Блок 0.5.

Загрузка библиотек

In [ ]:
# Standard python libraries
import os
import time

# Essential DS libraries
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import torch
import matplotlib.pyplot as plt
import seaborn as sns
#from pandas_profiling import ProfileReport


# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

Блок 0.5.

0.5.1 Загрузка тернировочного сета

In [265]:
# считываем тустовый и тренеровочный дата-сеты
train_data = pd.read_csv('data/train_data.csv')
test_data = pd.read_csv('data/test_data.csv')

In [270]:
train_data['car_vin'] = np.nan

In [271]:
train_data.head()

,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type,final_price
0,0,TOYOTA,NaN,Sedan,133000,2014,Automatic,4/5,Right-hand drive,Silver,Black,NaN,0,For Sale,3650.0
1,1,MERCEDES-BENZ,NaN,Sedan,24500,2010,Manual,4/5,Left wheel,Silver,Black,NaN,0,For Sale,6800.0
2,2,HYUNDAI,NaN,Hatchback,31000,2016,Tiptronic,2/3,Left wheel,Silver,Black,NaN,1,For Sale,6300.0
3,3,HYUNDAI,NaN,Jeep,115459,2015,Automatic,4/5,Left wheel,Blue,Black,NaN,1,For Sale,14488.0
4,4,TOYOTA,NaN,Jeep,18950,2019,Automatic,4/5,Left wheel,Black,NaN,NaN,1,For Sale,5000.0


In [ ]:
# просмотр информации о дата-сетах
train_data.info()

# видим два цифровых признака, которые необходимо посмотреть на искажения (выбросы): current_mileage и vehicle_year. Признаки проверим в обоих дата-сетах.

In [ ]:
# описание дата-сетов через describe
test_data.describe()

In [ ]:
train_data.describe()

In [ ]:
# В признаке current_mileage имеются нулевые значения, удалим их в тренировочной выборке

train_data = train_data[(train_data['current_mileage'] > 0)]

In [ ]:
# проверяем сколько строк удалилось
train_data.describe() # минус 1403 строки

In [ ]:
#строим коробчатую диаграмму

plt.figure(figsize=(20, 8))
boxplot = sns.boxplot(data=train_data, x='current_mileage')
boxplot.set_title('current_mileage')

plt.show()

# диаграмма показывает выбросы со значением более 2х млн км. В рамках знакомства с Kaggle не строим мажквартильный интервал по методу Тьюки.
# удилим записи с пробегом более 500 тыс. 

In [ ]:
train_data = train_data[(train_data['current_mileage'] <= 500000)]

In [ ]:
# проверяем сколько строк удалилось
train_data.describe() # минус еще 414 строк

In [ ]:
plt.figure(figsize=(20, 8))
boxplot = sns.boxplot(data=train_data, x='current_mileage')
boxplot.set_title('current_mileage')

plt.show()

# Диаграмма стали более информативной

In [ ]:
# работаем с vehicle_year

plt.figure(figsize=(20, 8))
boxplot = sns.boxplot(data=train_data, x='vehicle_year')
boxplot.set_title('vehicle_year')

plt.show()

# имеются выбросы в сторону старшинства автомобилей. Это обусловлено тем, что автомобили обновляются т.к. старый автомобиль требует больше бюджета для ремонта.
# также автомобили страше 50 лет считаются ретро-авто "антиквариат" и расчет цены идет по иным правилам.

In [ ]:
#удалим строки, где возраст автомобиля старше 50 лет, т.е. выпуск машины произошел до 1970 года
train_data = train_data[(train_data['vehicle_year'] >= 1970)] # минус 27 строк

In [ ]:
# повторное построение диаграммы
plt.figure(figsize=(20, 8))
boxplot = sns.boxplot(data=train_data, x='vehicle_year')
boxplot.set_title('vehicle_year')
plt.show()

# структура бокса сохранилась

0.5.2 Работаем с пропусками

In [ ]:
#Выявление процента пропусков по столбцам
cols_null_percent = train_data.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
display(cols_with_null)

#через тепловую карту
colors = ['blue', 'yellow'] 
fig = plt.figure(figsize=(20, 10))
cols = cols_with_null.index
ax = sns.heatmap(
    train_data[cols].isnull(),
    cmap=sns.color_palette(colors),
)

# Тепловая карта показала, что в четырех признаках имеются пропуски: car_vin, vehicle_interior_color, doors_cnt, vehicle_color.

In [ ]:
# vin содержит в себе закодированные сведения об автомобиле, для расшифровки его необходимо специально создавать алгоритм,
# пробуем удалить этот признак, т.к. в нём имеется много пропусков, также из-за уникальности значений признак неинформативен.
train_data.drop('car_vin', axis=1, inplace=True)

# также признак удалим и в тренирововсном сете
test_data.drop('car_vin', axis=1, inplace=True)

In [ ]:
# проверка: признаки car_vin
train_data.info()

In [ ]:
test_data.info()

Признак doors_cnt представлен в виде дроби и имеет тип object. Т.к. не могу прогнозировать как признак будет преобразован, то отдадим модели уже преобразованный признак.
В признаке указано количество дверей для пассажиров / количество дверей для пассажиров+дверь(крышка)багажника. У водителей можно встретить выражение: "Пятидверный седан", будем отталкиваться от этого и оставим количество дверей для пассажиров+дверь(крышка)багажника

In [ ]:
# оставляем последний символ строки
train_data['doors_cnt'] = train_data['doors_cnt'] = train_data.doors_cnt.str[-1]
train_data.head()

In [ ]:
# Также преобразуем и тестовый сет

test_data['doors_cnt'] = test_data['doors_cnt'] = test_data.doors_cnt.str[-1]
test_data.head()

In [ ]:
#пропуски в признаке doors_cnt заполним модой
mode_doors = train_data['doors_cnt'].mode() # mode = 5
train_data['doors_cnt'].fillna('mode_doors', inplace=True)
train_data.info()

In [ ]:
#пропуски в признаке doors_cnt заполним модой и в тестовой выборке
mode_doors = test_data['doors_cnt'].mode() # mode = 5
test_data['doors_cnt'].fillna('mode_doors', inplace=True)
test_data.info()

Также заполним модой нулевые значения в остальных признаках в тестовой и тренировочной выборках

In [ ]:
#пропуски в остальных признаках в выборках заполним модой
def fill_mode(df):
    """Функция DataFrame, где пропущенные значения заполнены модой соответствующего признака

    Args:
        df (DataFrame): таблицы csv содержащие данные для тренировки модели и предсказания цены на автомобили
    """
    mode_color = df['vehicle_color'].mode() # mode = Black
    df['vehicle_color'].fillna('mode_color', inplace=True)

    #пропуски в признаке vehicle_interior_color заполним модой
    mode_int_color = df['vehicle_interior_color'].mode() # mode = Black
    df['vehicle_interior_color'].fillna('mode_int_color', inplace=True)

    #пропуски в признаке vehicle_model заполним модой
    mode_model = df['vehicle_model'].mode() # mode = Black
    df['vehicle_model'].fillna('mode_model', inplace=True)

    #пропуски в признаке vehicle_manufacturer заполним модой
    mode_manufacture = df['vehicle_manufacturer'].mode() # mode = Black
    df['vehicle_manufacturer'].fillna('mode_manufacture', inplace=True)

    #пропуски в признаке vehicle_category заполним модой
    mode_category = df['vehicle_category'].mode() # mode = Black
    df['vehicle_category'].fillna('mode_category', inplace=True)

    #пропуски в признаке vehicle_gearbox_type заполним модой
    mode_gear = df['vehicle_gearbox_type'].mode() # mode = Black
    df['vehicle_gearbox_type'].fillna('mode_gear', inplace=True)

    #пропуски в признаке wheels заполним модой
    mode_weels = df['wheels'].mode() # mode = Black
    df['wheels'].fillna('mode_weels', inplace=True)
    return df
    
   
train_data = fill_mode(train_data)
test_data = fill_mode(test_data)

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
def correlation(df):
    """функция визуализирует тепловые карты корреляции по методам : Спирмена, Пирсена и Кендалла

    Args:
        df (DataFrame): таблица csv содержащая данные для предсказания цены на автомобили
    """
       
    plt.figure(figsize=(30, 8))
    plt.subplot(1, 3, 1)
    sns.heatmap(df.corr(method='spearman'), annot=True, linewidths=.5, cmap='CMRmap')
    plt.title ('Корреляция Spearman')

    plt.subplot(1, 3, 2)
    sns.heatmap(df.corr(method=('pearson')), annot=True, linewidths=.5, cmap='Blues')
    plt.title ('Корреляция Pearson')

    plt.subplot(1, 3, 3)
    sns.heatmap(df.corr(method=('kendall')), annot=True, linewidths=.5, cmap='CMRmap')
    plt.title ('Корреляция Kendall')
    
cor = correlation(train_data)

Из представленных тепловых карт видно, что корреляция признаков слабая. Из трех графов, первый -  корреляция Спирмена наиболее информативна. Почти все признаки слабо коррелируют между собой за исключением пар - car_leather_interior и current_mileage (возможно в определенные года выпускались только кожанные салоны), а также current_mileage и vehicle_year (это ожидаемо т.к. чем старше автомобиль, тем больше его пробег)<br>
Цена на автомобиль слабо, но явно коррелирует с годом выпуска и пробегом автомобиля, что естественно.

In [ ]:
# поиск дубликатов
def find_dupl(df):
    """Функция возвращает количество добликатов строк

    Args:
        df (DataFrame): таблица csv содержащая данные для предсказания цены на автомобили
    """
    #создаем список наименование колонок и удаляем индекс
    df_cols = list(df.columns)
    #diabetes_cols.remove('id')

    # создаем маску дубликатов и выводим количество повторяющихся строк
    mask=df.duplicated(subset=df_cols)
    df_dupl=df[mask]
    display(f'Число повторяющихся строк: {df_dupl.shape[0]}')
    
    
dupl = find_dupl(train_data)    

Функция показывает, что в датасете не имеется дубликатов строк. 
Но при удалении row_ID дубликаты появятся. Как скажется на модели удаление этого признака будет проверено опытным путем.

Проверяем признаки на неинформативность

In [ ]:
# выявим неинформатиные признаки в тренировочной выборке

def low_informative_feature(data):
    """Функция возвращает список неинформативных признаков

    Args:
        train_data (DataFrame): таблица csv содержащая данные для предсказания цены на автомобили
    """
    low_information_cols = []
    for col in data.columns:
        top_freq = data[col].value_counts(normalize=True).max()
        nunique_ratio = data[col].nunique() / data[col].count()
        if top_freq > 0.95:
            low_information_cols.append(col)
        if nunique_ratio > 0.95:
            low_information_cols.append(col)
    return low_information_cols
low_information_cols = low_informative_feature(train_data)
display(low_information_cols)

Пока пропустим признак row_ID, опытным путем выявим его вклад в обучение модели далее

In [ ]:
# просматриваем неинформативный признак на варинаты значений - их всего два
train_data['deal_type'].value_counts()

In [ ]:
# проверяем на нулевые значения
train_data['deal_type'].isnull().sum()

In [ ]:
# После проверки допускаем, что признак неинформативен и удаляем его из обеих выборок
train_data.drop('deal_type', axis=1, inplace=True)
test_data.drop('deal_type', axis=1, inplace=True)

In [ ]:
test_data.info()

Заключение:
Выявлены неинформативные признаки, показана корреляция признаков, преобразован признак, заполнены нулевые значения, удален неинформативный признак.<br>
Не применялись методы выявления выбросов, проверка на нормальность распределения признаков. Данные допущения позволены т.к. это превое знакомство с Kaggle и цель понять как работать в данном ресурсе.